In [1]:
from rdflib import Graph
from template_builder.utils import * 
from template_builder.namespaces import * 
from template_builder.get_completion import get_completion

In [2]:
g = Graph()
g.parse("https://brickschema.org/schema/1.4.3/Brick.ttl", format = "ttl")

<Graph identifier=Nef4d054c0bdb4bc8bad220d7733bb8b0 (<class 'rdflib.graph.Graph'>)>

In [3]:
query = """ SELECT DISTINCT ?brick_class ?brick_definition ?brick_parent WHERE {
    ?brick_class rdfs:subClassOf brick:Sensor ;
        skos:definition ?brick_definition .
    FILTER NOT EXISTS {
        ?brick_class2 rdfs:subClassOf brick:Sensor .
        ?brick_class rdfs:subClassOf ?brick_class2 .
    }
    FILTER NOT EXISTS {
        ?brick_class owl:deprecated true .
    }
    FILTER (!strstarts(str(?brick_class), "https://w3id.org/rec#")) 
    BIND (brick:Sensor AS ?brick_parent).
}"""

def copy_brick_data_to_csv(input_df, existing_csv_path, output_csv_path=None):
    """
    Extracts brick_class and brick_parent from input DataFrame and adds them
    to an existing CSV file as new rows, preserving all existing data.
    
    Parameters:
    -----------
    input_df : pandas.DataFrame
        DataFrame containing at least brick_class and brick_parent columns
    existing_csv_path : str
        Path to the existing CSV file
    output_csv_path : str, optional
        Path to save the resulting CSV. If None, overwrites the existing CSV.
    
    Returns:
    --------
    pandas.DataFrame
        The resulting DataFrame that was saved to CSV
    """
    # Ensure the input DataFrame has the required columns
    if 'brick_class' not in input_df.columns or 'brick_parent' not in input_df.columns:
        raise ValueError("Input DataFrame must contain 'brick_class' and 'brick_parent' columns")
    
    # Extract only the needed columns
    brick_data = input_df[['brick_class', 'brick_parent', 'brick_definition']].copy()
    
    # Read the existing CSV
    try:
        existing_df = pd.read_csv(existing_csv_path)
    except FileNotFoundError:
        print(f"Warning: Existing CSV file {existing_csv_path} not found. Creating a new one.")
        # Create a new DataFrame with expected columns
        existing_df = pd.DataFrame(columns=['brick_class', 'brick_parent', 'brick_definition', '223_class', 
                                           'quantitykind', 'medium', 'aspects'])
    
    # Get all columns from existing CSV
    all_columns = existing_df.columns.tolist()
    
    # Filter out brick_class values that already exist in the CSV
    existing_brick_classes = set(existing_df['brick_class']) if 'brick_class' in existing_df.columns else set()
    new_brick_data = brick_data[~brick_data['brick_class'].isin(existing_brick_classes)]
    
    if len(new_brick_data) == 0:
        print("No new brick_class entries to add. All provided brick_class values already exist in the CSV.")
        return existing_df
    
    # Ensure brick_data has all the columns from the existing CSV
    for col in all_columns:
        if col not in brick_data.columns:
            brick_data[col] = None  # Add missing columns with None values
    
    # Reorder columns to match the existing CSV
    brick_data = brick_data[all_columns]
    
    # Append the new data to the existing data
    result_df = pd.concat([existing_df, brick_data], ignore_index=True)
    
    # Save to CSV
    output_path = output_csv_path if output_csv_path else existing_csv_path
    result_df.to_csv(output_path, index=False)
    
    print(f"Data successfully appended to {output_path}")
    return result_df

# Example usage:
# Assuming you have a DataFrame called 'df' with brick_class and brick_parent columns
# and an existing CSV file at 'path/to/existing.csv'

# df = pd.DataFrame({
#     'brick_class': ['Class1', 'Class2', 'Class3'],
#     'brick_parent': ['Parent1', 'Parent2', 'Parent3'],
#     'other_column': [1, 2, 3]
# })
# 
# result = copy_brick_data_to_csv(df, 'path/to/existing.csv', 'path/to/output.csv')

In [4]:
brick_schema_df = copy_brick_data_to_csv(query_to_df(query, g), "mapping.csv")

Data successfully appended to mapping.csv


In [4]:
s223 = Graph()
s223.parse("https://open223.info/223p.ttl", format = "ttl")
bind_prefixes(s223)

In [6]:
# Get properties
prop_query = """ SELECT DISTINCT ?s223_class ?s223_definition WHERE {
    ?s223_class rdfs:subClassOf* s223:Property ;
    rdfs:comment ?s223_definition .
    FILTER NOT EXISTS {
        ?s223_class qudt:hasQuantityKind ?qk .
    }
}
"""
prop_df = query_to_df(prop_query, s223, remove_namespaces=True)
display(prop_df)
s223_properties = prop_df.to_csv(index=False)

,s223_class,s223_definition
0,Property,"An attribute, quality, or characteristic of a ..."
1,EnumerableProperty,An `EnumerableProperty` is a `Property` with a...
2,EnumeratedActuatableProperty,An `EnumeratedActuatableProperty` is a `Proper...
3,EnumeratedObservableProperty,An `EnumeratedObservableProperty` is a `Proper...
4,ObservableProperty,"The term ""observable"" implies that reading the..."
5,QuantifiableObservableProperty,This class is for instances of `QuantifiablePr...
6,ActuatableProperty,"The term ""actuatable"" implies that writing to ..."
7,QuantifiableActuatableProperty,This class is for instances of `QuantifiablePr...
8,QuantifiableProperty,This class is for quantifiable values that des...


In [31]:
# Get possible measurement locations 
loc_query = """ SELECT DISTINCT ?s223_class ?s223_definition WHERE {
    {
    ?s223_class rdfs:subClassOf+ s223:Connectable .
    }
    UNION
    {
    ?s223_class rdfs:subClassOf* s223:Connection .
    }
    UNION
    {
    ?s223_class rdfs:subClassOf+ s223:ConnectionPoint .
    }
    UNION
    {
    ?s223_class rdfs:subClassOf* s223:PhysicalSpace .
    }
    ?s223_class rdfs:comment ?s223_definition .

    # have to remove sensors, as they will be selected for all points
    FILTER NOT EXISTS {
        ?s223_class rdfs:subClassOf* s223:Sensor .
    }
    FILTER ( ?s223_class != s223:Equipment )
    # Getting rid of extensions
    FILTER(STRSTARTS (str(?s223_class), "http://data.ashrae.org/standard223#"))
}
"""
loc_df = query_to_df(loc_query, s223)
display(loc_df)
s223_loc = loc_df.to_csv(index=False)

,s223_class,s223_definition
0,s223:DomainSpace,A `DomainSpace` represents some portion of a `...
1,s223:Junction,A `Junction` is a modeling construct used when...
2,s223:Actuator,A piece of equipment that receives control sig...
3,s223:AirHandlingUnit,An assembly consisting of a fan or fans and ot...
4,s223:Boiler,"A closed, pressure vessel that uses fuel or el..."
...,...,...
89,s223:BidirectionalConnectionPoint,A `BidirectionalConnectionPoint` is a `Connect...
90,s223:OutletConnectionPoint,An `OutletConnectionPoint` is a `ConnectionPoi...
91,s223:InletConnectionPoint,An `InletConnectionPoint` is a `ConnectionPoin...
92,s223:PhysicalSpace,A `PhysicalSpace` is an architectural concept ...


In [21]:
for prefix, namespace in s223.namespaces():
    print(f"{prefix}: {namespace}")

brick: https://brickschema.org/schema/Brick#
csvw: http://www.w3.org/ns/csvw#
dc: http://purl.org/dc/elements/1.1/
dcat: http://www.w3.org/ns/dcat#
dcmitype: http://purl.org/dc/dcmitype/
dcterms: http://purl.org/dc/terms/
dcam: http://purl.org/dc/dcam/
doap: http://usefulinc.com/ns/doap#
foaf: http://xmlns.com/foaf/0.1/
geo: http://www.opengis.net/ont/geosparql#
odrl: http://www.w3.org/ns/odrl/2/
org: http://www.w3.org/ns/org#
prof: http://www.w3.org/ns/dx/prof/
prov: http://www.w3.org/ns/prov#
qb: http://purl.org/linked-data/cube#
schema: https://schema.org/
sh: http://www.w3.org/ns/shacl#
skos: http://www.w3.org/2004/02/skos/core#
sosa: http://www.w3.org/ns/sosa/
ssn: http://www.w3.org/ns/ssn/
time: http://www.w3.org/2006/time#
vann: http://purl.org/vocab/vann/
void: http://rdfs.org/ns/void#
wgs: https://www.w3.org/2003/01/geo/wgs84_pos#
owl: http://www.w3.org/2002/07/owl#
rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs: http://www.w3.org/2000/01/rdf-schema#
xsd: http://www.w3.

In [7]:
# Get Media
med_query = """ SELECT DISTINCT ?s223_class ?s223_definition WHERE {
    ?s223_class rdfs:subClassOf* s223:EnumerationKind-Substance ;
    rdfs:comment ?s223_definition .
    FILTER NOT EXISTS {
        {
            ?s223_class rdfs:subClassOf+ s223:Electricity-DC .
        }
        UNION
        {
            ?s223_class rdfs:subClassOf+ s223:Electricity-AC .
        }
    }
}
"""
media_df = query_to_df(med_query, s223)
display(media_df)
s223_media = media_df.to_csv(index=False)

,s223_class,s223_definition
0,s223:EnumerationKind-Substance,This class has enumerated subclasses of the su...
1,s223:Substance-Soot,`Substance-Soot`
2,s223:Substance-Particulate,This class has enumerated subclasses of partic...
3,s223:Particulate-PM1.0,Particulate-PM1.0
4,s223:Particulate-PM10.0,Particulate-PM10.0
5,s223:Particulate-PM2.5,Particulate-PM2.5
6,s223:Substance-Medium,This class has enumerated subclasses of a phys...
7,s223:Medium-Mix,`Medium-Mix`
8,s223:Mix-PowerAndSignal,Mix-Power and signal
9,s223:PowerAndSignal-PoE,This class has enumerated subclasses of Power ...


In [188]:
# Get Aspects
asp_query = """ SELECT DISTINCT ?s223_class ?s223_definition WHERE {
    ?s223_class rdfs:subClassOf* s223:EnumerationKind .
    ?s223_class rdfs:comment ?s223_definition .
  
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-Substance .
    }
    # Excluding all the voltages
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-Numerical .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-ElectricalPhaseIdentifier .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-ElectricalVoltagePhases .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-DayOfWeek .
    }
}
"""
asp_df = query_to_df(asp_query, s223)
display(asp_df)
s223_aspects = asp_df.to_csv(index=False)

,s223_class,s223_definition
0,s223:EnumerationKind,This is the encapsulating class for all Enumer...
1,s223:EnumerationKind-Occupancy,This class has enumerated subclasses of occupa...
2,s223:Occupancy-Occupied,Occupied
3,s223:Occupancy-Unoccupied,Unoccupied
4,s223:Occupancy-Motion,This class has enumerated subclasses indicatin...
...,...,...
86,s223:Aspect-Threshold,The threshold value of something characterized...
87,s223:Aspect-WetBulb,The associated `Property` is a WetBulb tempera...
88,s223:Aspect-Effectiveness,This class enumerates the possible states of e...
89,s223:Effectiveness-Active,Active


In [213]:
# Get EnumerationKind
ek_query = """ SELECT DISTINCT ?s223_class ?s223_definition WHERE {
    ?s223_class rdfs:subClassOf* s223:EnumerationKind .
    ?s223_class rdfs:comment ?s223_definition .
  
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-Substance .
    }
    # Excluding all the voltages
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-Numerical .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-ElectricalPhaseIdentifier .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-ElectricalVoltagePhases .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-DayOfWeek .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-Aspect .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-Role .
    }
    FILTER NOT EXISTS {        
        ?s223_class rdfs:subClassOf* s223:EnumerationKind-Domain .
    }
}
"""
ek_df = query_to_df(ek_query, s223)
display(ek_df)
s223_eks = ek_df.to_csv(index=False)

,s223_class,s223_definition
0,s223:EnumerationKind,This is the encapsulating class for all Enumer...
1,s223:EnumerationKind-Occupancy,This class has enumerated subclasses of occupa...
2,s223:Occupancy-Occupied,Occupied
3,s223:Occupancy-Unoccupied,Unoccupied
4,s223:Occupancy-Motion,This class has enumerated subclasses indicatin...
5,s223:Motion-False,`Motion-False`
6,s223:Motion-True,`Motion-True`
7,s223:Occupancy-Presence,This class has enumerated subclasses indicatin...
8,s223:Presence-False,`Presence-False`
9,s223:Presence-True,`Presence-True`


In [199]:
brick_class = brick_schema_df['brick_class'][0]
definition = brick_schema_df['brick_definition'][0]
system_prompt = """"""
prompt = f"""
Determine what s223_class the brick_class should be, based on its name and definition.
the possible s223 classes are <s223_properties>{s223_properties}</s223_properties> 

Only return the s223_class. Do not return any other information.

brick_class: {brick_class}
definition: {definition}
"""
s223_class = get_completion(prompt, system_prompt)
print(f"brick_class: {brick_class}")
print(f"definition: {definition}")
print(f"s223_class: {s223_class}")


brick_class: brick:Contact_Sensor
definition: Senses or detects contact, such as for determining if a door is closed.
s223_class: s223:EnumeratedObservableProperty


In [207]:
# What are the valid quantitykinds
# building_models = Graph()
# building_models.parse("https://models.open223.info/compiled/nist-bdg1-1.ttl", format = "ttl")
# building_models.parse("https://models.open223.info/pnnl-bdg1-2.ttl", format = "ttl")
# building_models.parse("https://models.open223.info/pnnl-bdg2-1.ttl", format = "ttl")
# building_models.parse("https://models.open223.info/lbnl-bdg3-1.ttl", format = "ttl")
# building_models.parse("https://models.open223.info/lbnl-bdg4-1.ttl", format = "ttl")


# query = """ SELECT DISTINCT ?quantitykind WHERE {
#     ?s qudt:hasQuantityKind ?quantitykind .
# }"""
# quantitykind_df = query_to_df(query, building_models)
# display(quantitykind_df)

# Completed by hand

In [210]:
with open("quantitykinds.csv", "r") as f:
    quantitykinds = f.read()

In [214]:
prompt = f"""
Determine what quantitykind or enumerationkind the brick_class should be, based on its name and definition.
the possible quantitykinds are <quantitykinds>{quantitykinds}</quantitykinds> 
the possible enumerationkinds are <s223_eks>{s223_eks}</s223_eks>
Only return the quantitykind or enumerationkind. Do not return any other information.

brick_class: {brick_class}
definition: {definition}
"""
s223_class = get_completion(prompt, system_prompt)
print(f"brick_class: {brick_class}")
print(f"definition: {definition}")
print(f"s223_class: {s223_class}")


brick_class: brick:Contact_Sensor
definition: Senses or detects contact, such as for determining if a door is closed.
s223_class: s223:Binary-Position


In [217]:
prompt = f"""
Determine what medium the brick_class should be associated with, based on its name and definition.
the possible media are <media>{s223_media}</media> 
Only return the quantitykind or enumerationkind. Do not return any other information.

If there is no sensible medium, return None.

brick_class: {brick_class}
definition: {definition}
"""
s223_class = get_completion(prompt, system_prompt)
print(f"brick_class: {brick_class}")
print(f"definition: {definition}")
print(f"s223_class: {s223_class}")


brick_class: brick:Contact_Sensor
definition: Senses or detects contact, such as for determining if a door is closed.
s223_class: None


In [ ]:
prompt = f"""
Determine what aspects the brick_class should be associated with, based on its name and definition.
the possible aspects are <aspects>{s223_aspects}</aspects> 
Only return the quantitykind or enumerationkind. Do not return any other information.

If there is no sensible medium, return None.

brick_class: {brick_class}
definition: {definition}
"""
s223_class = get_completion(prompt, system_prompt)
print(f"brick_class: {brick_class}")
print(f"definition: {definition}")
print(f"s223_class: {s223_class}")


brick_class: brick:Contact_Sensor
definition: Senses or detects contact, such as for determining if a door is closed.
s223_class: s223:Binary-Position


In [23]:
from pydantic import BaseModel, Field
import openai
import json
import yaml
# Define your Pydantic model for a single word response
class SingleWordResponse(BaseModel):
    word: str = Field(description="A single word that represents the answer")

with open('/Users/lazlopaul/Desktop/cborg/api_key.yaml', 'r') as file:
    config = yaml.safe_load(file)
    API_KEY = config['key']
    BASE_URL = config['base_url']

client = openai.OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# Create your prompt with thinking enabled
response = client.chat.completions.create(
    model="lbl/llama",  # or another appropriate model
    messages=[
        {"role": "system", "content": "You are an assistant that provides a a 1 sentence answer after careful consideration."},
        {"role": "user", "content": f"""Determine what quantitykind or enumerationkind the brick_class should be, based on its name and definition.
        the possible quantitykinds are <quantitykinds>{pd.read_csv("template_builder/data/quantitykinds.csv").to_csv(index=False)}</quantitykinds> 

        brick_class: Temperature_Sensor
        definition: A sensor measuring temperature"""}
    ],
    temperature=0.7,
    # The key part - use function calling to get structured output
    functions=[
        {
            "name": "provide_single_word",
            "description": "Provide a single word answer after thinking through the options",
            "parameters": SingleWordResponse.model_json_schema()
        }
    ],
    function_call={"name": "provide_single_word"}
)

# Extract the thinking and structured response
thinking = response.choices[0].message.content
function_call = response.choices[0].message.function_call

# Parse the function arguments as JSON
parsed_response = json.loads(function_call.arguments)
single_word = parsed_response["word"]

print("Thinking:", thinking)
print("Final answer:", parsed_response)
print("Just the word:", single_word)

AttributeError: 'NoneType' object has no attribute 'arguments'

In [24]:
function_call

In [25]:
from devtools import pprint
pprint(response)

ChatCompletion(
    id='chatcmpl-ba5a83eafd1046a58f903c00d3742f3b',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=(
                    'Based on the name "Temperature_Sensor" and its definition "A sensor measuring temperature", the `'
                    'brick_class` should be of quantitykind `Temperature`.'
                ),
                refusal=None,
                role='assistant',
                annotations=None,
                audio=None,
                function_call=None,
                tool_calls=None,
            ),
        ),
    ],
    created=1745359720,
    model='lbl/llama',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=32,
        prompt_tokens=146,
        total_tokens=178,
        completion_tokens_details=None,
        prompt_tokens_details

In [ ]:
def get_single_word_response(prompt):
    response = client.chat.completions.create(
        model="lbl/llama",  # or another model of your choice
        messages=[
            {"role": "system", "content": "Think carefully about the question."},
            {"role": "user", "content": prompt}
        ],
        tools=[{  # Note: newer versions use 'tools' instead of 'functions'
            "type": "function",
            "function": {
                "name": "provide_single_word",
                "description": "Provide a single word as the answer",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "word": {
                            "type": "string",
                            "description": "The single word answer"
                        }
                    },
                    "required": ["word"]
                }
            }
        }],
        tool_choice={"type": "function", "function": {"name": "provide_single_word"}}  # Force use of this tool
    )
    
    # Extract the single word from the function call
    tool_call = response.choices[0].message.tool_calls[0]
    function_args = json.loads(tool_call.function.arguments)
    
    return function_args.get("word")

# Example usage
prompt = "What's the capital of France? Think carefully."
result = get_single_word_response(prompt)
print(result)  # Should output: "Paris"

Paris


In [28]:
prop_query = """ SELECT DISTINCT ?s223_class ?s223_definition WHERE {
        ?s223_class rdfs:subClassOf* s223:Property ;
        rdfs:comment ?s223_definition .
        FILTER NOT EXISTS {
            ?s223_class qudt:hasQuantityKind ?qk .
        }
        FILTER NOT EXISTS {
            ?s223_class2 rdfs:subClassOf ?s223_class .
        }
        FILTER(STRSTARTS (str(?s223_class), \"http://data.ashrae.org/standard223#\"))
    }
    """
prop_df = query_to_df(prop_query, s223, remove_namespaces=True)

In [29]:
prop_df

,s223_class,s223_definition
0,EnumeratedActuatableProperty,An `EnumeratedActuatableProperty` is a `Proper...
1,EnumeratedObservableProperty,An `EnumeratedObservableProperty` is a `Proper...
2,QuantifiableObservableProperty,This class is for instances of `QuantifiablePr...
3,QuantifiableActuatableProperty,This class is for instances of `QuantifiablePr...
